In [1]:
from sktime.datasets import load_from_tsfile_to_dataframe
import numpy as np
import pandas as pd

def load_from_tsfile(file_path, return_y=True):
    X, y = load_from_tsfile_to_dataframe(file_path)
    X = pd.DataFrame({i: pd.Series(x) for i, x in enumerate(X.iloc[:, 0])})
    if return_y:
        return X, y
    else:
        return X


# Test the function
dname= 'PEMS-SF'
file_path = f'./datasets/{dname}/{dname}_TRAIN.ts'
X, y = load_from_tsfile(file_path)
print(X.shape, y.shape)



C:\Users\ui572274\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 10.0.1 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
c:\Users\ui572274\.conda\envs\sd\lib\site-packages\sktime\datasets\_readers_writers\ts.py:594: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["dim_" + str(dim)] = instance_list[dim]
c:\Users\ui572274\.conda\envs\sd\lib\site-packages\sktime\datasets\_readers_writers\ts.py:594: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

(144, 267) (267,)


In [2]:
file_path = f'./datasets/{dname}/{dname}_TEST.ts'
X_test, y_test = load_from_tsfile(file_path)
print(X_test.shape, y_test.shape)

c:\Users\ui572274\.conda\envs\sd\lib\site-packages\sktime\datasets\_readers_writers\ts.py:594: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["dim_" + str(dim)] = instance_list[dim]
c:\Users\ui572274\.conda\envs\sd\lib\site-packages\sktime\datasets\_readers_writers\ts.py:594: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["dim_" + str(dim)] = instance_list[dim]
c:\Users\ui572274\.conda\envs\sd\lib\site-packages\sktime\datasets\_readers_writers\ts.py:594: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

(144, 173) (173,)


In [3]:
X_train  = X.transpose()

In [4]:
X_test = X_test.transpose()

In [5]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and return encoded labels
y = label_encoder.fit_transform(y)
y_test = label_encoder.fit_transform(y_test)


In [6]:
num_classes = len(np.unique(y_test))

num_classes

7

In [9]:
import lightgbm as lgb
import optuna
from sklearn.metrics import accuracy_score

def objective(trial):
    param = {
        'objective': 'multiclass',  # or 'multiclass' for multi-class classification
        'metric': 'multi_logloss',  # or 'multi_logloss' for multi-class classification

        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
    }

    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train, y)
    preds = gbm.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy



In [10]:

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)



[I 2024-02-08 17:39:59,097] A new study created in memory with name: no-name-cf04d2b3-c596-4b70-8c26-548543f897f9
[I 2024-02-08 17:39:59,677] Trial 0 finished with value: 0.7514450867052023 and parameters: {'num_leaves': 163, 'learning_rate': 0.14786178286950508, 'n_estimators': 525}. Best is trial 0 with value: 0.7514450867052023.
[I 2024-02-08 17:40:02,389] Trial 1 finished with value: 0.20809248554913296 and parameters: {'num_leaves': 97, 'learning_rate': 3.657978758308325e-05, 'n_estimators': 611}. Best is trial 0 with value: 0.7514450867052023.
[I 2024-02-08 17:40:06,225] Trial 2 finished with value: 0.11560693641618497 and parameters: {'num_leaves': 10, 'learning_rate': 3.5191773920450777e-06, 'n_estimators': 963}. Best is trial 0 with value: 0.7514450867052023.
[I 2024-02-08 17:40:09,731] Trial 3 finished with value: 0.11560693641618497 and parameters: {'num_leaves': 165, 'learning_rate': 3.2413463724727126e-08, 'n_estimators': 838}. Best is trial 0 with value: 0.751445086705202

Number of finished trials: 100
Best trial: {'num_leaves': 190, 'learning_rate': 0.16217231020965087, 'n_estimators': 626}
